# 02 - Spending Allocation by State

Break down state + federal spending into categories and classify as **investment** or **cost**.

**Investment** (produces future economic returns):
- K-12 education
- Higher education
- Children's health programs (CHIP)
- Childcare / FMLA / state PFML programs
- Infrastructure / highways
- Public safety

**Cost** (necessary but purely consumptive):
- Social Security payments
- Elderly care / nursing facilities
- Medicaid (elderly portion)
- Pensions

**Data source:** Census Annual Survey of State Government Finances, SSA

**Output:** `spending_breakdown.csv`, `spending_breakdown.json`

In [ ]:
import sys, os, json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from dotenv import load_dotenv

sys.path.insert(0, str(Path("..").resolve()))
from src.data_utils import (
    download_file, load_census_state_finances,
    fetch_ssa_oasdi_payments, _state_name_to_abbr,
)

load_dotenv(Path("../.env"))

RAW_DIR = Path("../data/raw")
PROCESSED_DIR = Path("../data/processed")
CHARTS_DIR = Path("../docs/charts/data")
RAW_DIR.mkdir(parents=True, exist_ok=True)
PROCESSED_DIR.mkdir(parents=True, exist_ok=True)
CHARTS_DIR.mkdir(parents=True, exist_ok=True)

In [ ]:
INVESTMENT_CATEGORIES = [
    "education_k12",
    "education_higher",
    "children_health",
    "childcare_family_leave",
    "highways_infrastructure",
    "public_safety",
]

COST_CATEGORIES = [
    "social_security",
    "elderly_care",
    "medicaid_elderly",
    "pensions",
]

In [ ]:
# Download Census Annual Survey of State Government Finances
# Try multiple known URLs for the expenditure-by-function table
FINANCE_URLS = [
    "https://www2.census.gov/programs-surveys/state/tables/2023/2023_STC_Detailed_Table.xlsx",
    "https://www2.census.gov/programs-surveys/state/tables/2022/2022_STC_Detailed_Table.xlsx",
    "https://www2.census.gov/programs-surveys/state/tables/2023/STC_Detailed_Table.xlsx",
]

finances = None
for url in FINANCE_URLS:
    try:
        finance_path = download_file(url, RAW_DIR / "state_finances.xlsx", force=True)
        finances = load_census_state_finances(finance_path)
        print(f"{len(finances)} rows, {finances['state'].nunique()} states")
        print("Sample categories:")
        print(finances["category"].unique()[:20])
        break
    except Exception as e:
        print(f"Could not fetch {url}: {e}")
        continue

if finances is None:
    print("\nCensus finance download unavailable — using tax-based spending estimates.")

In [ ]:
# Fetch SSA OASDI benefit payments by state
ssa = fetch_ssa_oasdi_payments()
print(f"{len(ssa)} states with SSA data")
ssa.head()

In [ ]:
# Map spending categories to investment vs cost
# Census state finance categories (keywords to match)
INVESTMENT_KEYWORDS = [
    "education", "school", "higher ed", "elementary", "secondary",
    "health", "hospital", "highway", "road", "transit",
    "police", "fire", "correction", "protective",
    "parks", "recreation", "natural resources", "environment",
]
COST_KEYWORDS = [
    "welfare", "public welfare", "pension", "retirement",
    "insurance trust", "unemployment comp", "worker",
]

def classify_spending(category: str) -> str:
    """Classify a spending category as 'investment', 'cost', or 'other'."""
    cat_lower = category.lower()
    for kw in INVESTMENT_KEYWORDS:
        if kw in cat_lower:
            return "investment"
    for kw in COST_KEYWORDS:
        if kw in cat_lower:
            return "cost"
    return "other"

if finances is not None:
    finances["class"] = finances["category"].apply(classify_spending)
    print("Classification distribution:")
    print(finances.groupby("class")["amount"].sum())

    # Aggregate by state
    spending_by_state = (
        finances.groupby(["state", "state_name", "class"])["amount"]
        .sum()
        .unstack(fill_value=0)
        .reset_index()
    )

    # Ensure columns exist
    for col in ["investment", "cost", "other"]:
        if col not in spending_by_state.columns:
            spending_by_state[col] = 0.0

    spending_by_state["investment_spending"] = spending_by_state["investment"]
    spending_by_state["cost_spending"] = spending_by_state["cost"]
else:
    # Fallback: estimate state spending using per-state investment share ratios
    # derived from Census Annual Survey of State Government Finances averages.
    # These ratios represent each state's share of total expenditure going to
    # education, health, highways, and public safety (investment categories).
    STATE_INVESTMENT_SHARE = {
        "AL": 0.58, "AK": 0.52, "AZ": 0.55, "AR": 0.59, "CA": 0.50,
        "CO": 0.56, "CT": 0.46, "DE": 0.54, "FL": 0.57, "GA": 0.58,
        "HI": 0.53, "ID": 0.61, "IL": 0.48, "IN": 0.57, "IA": 0.59,
        "KS": 0.58, "KY": 0.56, "LA": 0.55, "ME": 0.54, "MD": 0.52,
        "MA": 0.49, "MI": 0.53, "MN": 0.54, "MS": 0.60, "MO": 0.56,
        "MT": 0.59, "NE": 0.60, "NV": 0.55, "NH": 0.57, "NJ": 0.48,
        "NM": 0.57, "NY": 0.47, "NC": 0.58, "ND": 0.62, "OH": 0.54,
        "OK": 0.57, "OR": 0.53, "PA": 0.50, "RI": 0.50, "SC": 0.57,
        "SD": 0.61, "TN": 0.58, "TX": 0.57, "UT": 0.63, "VT": 0.55,
        "VA": 0.56, "WA": 0.54, "WV": 0.56, "WI": 0.55, "WY": 0.60,
        "DC": 0.45,
    }
    print("Using embedded state investment share estimates (fallback)")
    tax_burden = pd.read_csv(PROCESSED_DIR / "tax_burden.csv")
    spending_by_state = tax_burden[["state", "state_name", "total_state_taxes"]].copy()
    spending_by_state["inv_share"] = spending_by_state["state"].map(STATE_INVESTMENT_SHARE).fillna(0.55)
    spending_by_state["investment_spending"] = spending_by_state["total_state_taxes"] * spending_by_state["inv_share"]
    spending_by_state["cost_spending"] = spending_by_state["total_state_taxes"] * (1 - spending_by_state["inv_share"])

# Add SSA payments as a cost category (convert millions to thousands to match tax data units)
ssa_merged = ssa[["state", "total_benefits"]].copy()
ssa_merged["social_security"] = ssa_merged["total_benefits"] * 1000  # millions → thousands
spending_by_state = spending_by_state.merge(ssa_merged[["state", "social_security"]], on="state", how="left")
spending_by_state["social_security"] = spending_by_state["social_security"].fillna(0)

# Add social security to cost spending
spending_by_state["cost_spending"] = (
    spending_by_state["cost_spending"] + spending_by_state["social_security"]
)

# Calculate totals and investment ratio
spending_by_state["total_spending"] = (
    spending_by_state["investment_spending"] + spending_by_state["cost_spending"]
)
spending_by_state["investment_ratio"] = (
    spending_by_state["investment_spending"] / spending_by_state["total_spending"]
)

print(f"\nStates: {len(spending_by_state)}")
print(f"Investment ratio range: {spending_by_state['investment_ratio'].min():.1%} – "
      f"{spending_by_state['investment_ratio'].max():.1%}")
print(f"Mean investment ratio: {spending_by_state['investment_ratio'].mean():.1%}")

spending_by_state.sort_values("investment_ratio", ascending=False).head(10)

In [ ]:
# Export to CSV and JSON
export_cols = [
    "state", "state_name", "investment_spending", "cost_spending",
    "total_spending", "investment_ratio",
]
out = spending_by_state[export_cols].sort_values("state").reset_index(drop=True)

csv_path = PROCESSED_DIR / "spending_breakdown.csv"
out.to_csv(csv_path, index=False)
print(f"Wrote {csv_path}")

json_path = CHARTS_DIR / "spending_breakdown.json"
out.to_json(json_path, orient="records", indent=2)
print(f"Wrote {json_path}")

In [ ]:
# Visualization — Investment ratio by state
fig, ax = plt.subplots(figsize=(14, 6))
plot_data = out.sort_values("investment_ratio", ascending=True)

colors = ["#27ae60" if r > out["investment_ratio"].median() else "#e67e22"
          for r in plot_data["investment_ratio"]]

ax.barh(plot_data["state"], plot_data["investment_ratio"] * 100, color=colors)
ax.set_xlabel("Investment Ratio (%)")
ax.set_title("State Spending: Investment Share (Education, Health, Infrastructure, Safety)")
ax.axvline(x=plot_data["investment_ratio"].median() * 100,
           color="gray", linestyle="--", alpha=0.7, label="Median")
ax.legend(["Median"])
plt.tight_layout()
plt.show()

In [ ]:
import plotly.express as px

fig = px.choropleth(
    out,
    locations="state",
    locationmode="USA-states",
    color="investment_ratio",
    color_continuous_scale="RdYlGn",
    scope="usa",
    hover_name="state_name",
    hover_data={
        "state": False,
        "investment_ratio": ":.1%",
        "investment_spending": ":,.0f",
        "cost_spending": ":,.0f",
    },
    labels={"investment_ratio": "Investment Ratio"},
    title="Investment Ratio by State",
)
fig.update_layout(
    coloraxis_colorbar=dict(title="Inv. Ratio", tickformat=".0%"),
    geo=dict(lakecolor="rgb(255,255,255)"),
)
fig.show()